
# Semantic Kernel - The Kernel 

Semantic Kernel is an open-source SDK designed for developers to easily integrate AI agents into their existing systems, facilitating automation and enhancing user experiences. This SDK bridges the gap between AI capabilities and practical application, making it straightforward to bring AI insights into actionable processes.




## Core Features
- AI Service Integration: 
It offers a uniform interface for connecting with various AI services like OpenAI, Azure OpenAI, HuggingFace, and Dall-e. This ensures developers can easily integrate different AI technologies without worrying about compatibility issues.

- Extending AI to Real-World Actions: 
Beyond mere integration, the SDK allows for the execution of native code and actions based on AI outputs, supporting complex workflows through input and output chaining.Unified Interface to AI services
Extend AI ablility to exteral world


## Kernel
Kernel is the center component of an AI applciation that manages all AI resources.
With regiestered AI models, services (logging, telemetry) and plugings, 
the kernel will handle all the execution details: 

1. Select the best AI service to run the prompt.

2. Build the prompt using the provided prompt template.

3. Send the prompt to the AI service.

4. Receive and parse the response.

5. Pipeline, chain all executions


## Example: A simple agent

In the following code, we will demonstrate the minimum steps for building an AI agent that follows user's instructions to complete tasks on a local system.

1. register an AI service. In this example, it is an Azure OpenAI "GPT-4" deployment.

2. add two plugings: TimePlugin and LightPlugin.

 - [TimePlugin](https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/out-of-the-box-plugins?tabs=Csharp) is an out-of-the-box plugin provided by Semantic Kernel, 
 
 - [LightPlugin]((./plugins/LightPlugin.cs).) is a plugin written in native code. 
 
    This plugin contains group of function that controls a dummy lighting system. Check its implementation code [here](./plugins/LightPlugin.cs)
 
Under the hood, Semantic Kernel handles all the heavy lifting by constructing requests to the AI service, parsing the responses, invoking local functions, and chaining the messages.

### Step 1: Importing the required libraries

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha"
#!import config/Settings.cs 
#!import lib/Usings.cs
#!import lib/Utilities.cs
#!import plugins/LightPlugin.cs

Installed Packages Microsoft.SemanticKernel, 1.3.1 Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha

### Step 2: build the kernel

In [4]:
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();

//register ai serivces
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

//register plugins
#pragma warning disable SKEXP0050
builder.Plugins.AddFromType<TimePlugin>();
builder.Plugins.AddFromType<LightPlugin>();

var kernel = builder.Build();


### Step 3: send a request to the agent. 

We have AI service and plugins configured in prevous step.

Now lets make a request to that agent.

In [5]:
var input = "Turn on the lights if it is night time.";

Console.WriteLine($"[User] > {input}");

ChatHistory history = [];
history.AddSystemMessage("You are a agent that automates building lights control. Only reponse to requests that are related to lights control. You can also help with time related questions. If any other request is made, please response 'Sorry, I don't know.");
history.AddUserMessage(input);

// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Enable auto function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

Console.WriteLine("[Assistant] > " + result.Content);


[User] > Turn on the lights if it is night time.
[Light is now on]
[Assistant] > The lights are now turned on.


If everything is set up correctly, we should see the output tells us the dummy lighs have been turned on/off. 

## What has happened under the hood

<font >

Despite receiving only one user input, the kernel efficiently coordinated a series of task executions.

First, it selected the default AI service to transmit the prompt. Alongside the prompt text, it included function metadata to inform the AI service about available functions.

Subsequently, it parsed the AI service response to initiate function calls and chain executions seamlessly.

</font> 

Let's closely examine the chat history containing all intermediate message exchanges between the AI service and the kerne"l.

In [6]:
history.Display()

[ You are a agent that automates building lights control. Only reponse to requests that are related to lights control. You can also help with time related questions. If any other request is made, please response 'Sorry, I don't know., Turn on the lights if it is night time., , 17, off, , on, The lights are now turned on. ] Count 8 (values) index type value 0 Microsoft.SemanticKernel.ChatMessageContent You are a agent that automates building lights control. Only reponse to requests that are related to lights control. You can also help with time related questions. If any other request is made, please response 'Sorry, I don't know. Role system Label system Content You are a agent that automates building lights control. Only reponse to requests that are related to lights control. You can also help with time related questions. If any other request is made, please response 'Sorry, I don't know. Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 InnerContent <null> ModelId <null> Metadata <null> 1 Microsoft.SemanticKernel.ChatMessageContent Turn on the lights if it is night time. Role user Label user Content Turn on the lights if it is night time. Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 InnerContent <null> ModelId <null> Metadata <null> 2 Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIChatMessageContent ToolCalls index value 0 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name TimePlugin-HourNumber Arguments {} Id call_M5n5nSJCz0OKZmd2rURm80B0 1 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name LightPlugin-GetState Arguments {} Id call_NVXVhP0gwHEPPk1xON9tNVrl Role assistant Label assistant Content <null> Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 InnerContent Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls index value 0 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name TimePlugin-HourNumber Arguments {} Id call_M5n5nSJCz0OKZmd2rURm80B0 1 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name LightPlugin-GetState Arguments {} Id call_NVXVhP0gwHEPPk1xON9tNVrl FunctionCall <null> AzureExtensionsContext <null> ModelId GPT-4 Metadata key type value Id System.String chatcmpl-8wIEZ1bvxZdHN3kYR8wJIiVri7UfB Created System.DateTimeOffset 2024-02-25 23:30:07Z PromptFilterResults System.Collections.Generic.List<Azure.AI.OpenAI.ContentFilterResultsForPrompt> index value 0 Azure.AI.OpenAI.ContentFilterResultsForPrompt PromptIndex 0 ContentFilterResults Azure.AI.OpenAI.ContentFilterResultDetailsForPrompt Sexual Azure.AI.OpenAI.ContentFilterResult Violence Azure.AI.OpenAI.ContentFilterResult Hate Azure.AI.OpenAI

  Semantic Kernel abstracts away complex AI service integration logic and provides a uniform interface to communicate with different AI providers.

  With its plugin system, we can extend AI cabilities by calling external tools.

  Semantic Kernel also manages the history and orchestrates function executions.

## Your Turn: 
1. ask the AI service to explain its thought process.

In [7]:
history.AddUserMessage("explain your thought process of turning on/off the lights");
result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

result.Display()

When deciding whether to turn on or off the lights, I follow a structured thought process that takes into account both the current time and the existing state of the lights. Here's how I approach the situation:\n\n1. **Determine the Current Hour:** I use the `TimePlugin-HourNumber` tool to get the c... ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) Role assistant Label assistant Content When deciding whether to turn on or off the lights, I follow a structured thought process that takes into account both the current time and the existing state of the lights. Here's how I approach the situation:

1. **Determine the Current Hour:** I use the `TimePlugin-HourNumber` tool to get the current hour in 24-hour format. This helps me understand what time of day it is, which is crucial for deciding whether it's likely to be dark outside.

2. **Check Current Light State:** Simultaneously, I use the `LightPlugin-GetState` tool to check whether the lights are currently on or off. Knowing the existing state of the lights is important, as it influences whether any action needs to be taken.

3. **Consider the Time of Day in Context:** I interpret the current hour to determine if it is night time. Typically, I consider it night time if the hour is after sunset or before sunrise, which, for simplicity, I might approximate to a range such as between 18:00 (6:00 PM) and 06:00 (6:00 AM) unless specific sunset and sunrise times are known.

4. **Decide on the Action to Take:** 
 - If it's night time and the lights are off, I decide to turn on the lights.
 - If it's during the day or the lights are already on, I assess whether turning them off is necessary based on the initial request.
 
5. **Execute the Action:** Once I've determined that action needs to be taken (for example, turning on the lights), I execute the appropriate command using the `LightPlugin-ChangeState` tool, changing the state of the lights as required.

In your specific request, I determined it was night time (17:00, or 5:00 PM), and the lights were currently off. Given your instruction to "Turn on the lights if it is night time," and considering the conditions were met (it was night time, and the lights were off), I proceeded to turn on the lights by setting their state to 'on'. My actions always aim to fulfill the request accurately, ensuring the lights are controlled as desired based on time and their current state. Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 InnerContent Azure.AI.OpenAI.ChatResponseMessage Role assistant Content When deciding whether to turn on or off the lights, I follow a structured thought process that takes into account both the current time and the existing state of the lights. Here's how I approach the situation:

1. **Determine the Current Hour:** I use the `TimePlugin-HourNumber` tool to get the current hour in 24-hour format. This helps me understand what time of day it is, which is crucial for deciding whether it's likely to be dark outside.

2. **Check Current Light State:** Simultaneously, I use the `LightPlugin-GetState` tool to check whether the lights are currently on or off. Knowing the existing state of the lights is important, as it influences whether any action needs to be taken.

3. **Consider the Time of Day in Context:** I interpret the current hour to determine if it is night time. Typically, I consider it night time if the hour is after sunset or before sunrise, which, for simplicity, I might approximate to a range such as between 18:00 (6:00 PM)

2: try write a block of pseudo code to have some thought logic the AI service used

In [8]:
history.AddUserMessage("describe the logic to control lignts on/off according to current day/night. Use pseduo code.");
result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

result.Content.DisplayAs("text/markdown")

To control lights on and off according to whether it's currently day or night, we can use a simple logic that checks the current time against standard times for sunrise and sunset. Note that in a real scenario, these times can change throughout the year and may need to be dynamically determined based on the geographical location. However, for the sake of simplicity, we will use fixed times. Here's the pseudo code to achieve this:

```plaintext
function controlLightsBasedOnDayNight() {
    // Define fixed times for sunrise and sunset
    sunrise_hour = 6  // Assuming sunrise at 6:00 AM
    sunset_hour = 18  // Assuming sunset at 6:00 PM

    // Get the current hour in 24-hour format
    current_hour = getCurrentHour()

    // Get the current state of the lights (true for on, false for off)
    light_state = getLightState()

    // Check if it's night time
    if current_hour < sunrise_hour or current_hour >= sunset_hour {
        // It's night time
        
        if not light_state {
            // If lights are off, turn them on
            turnLightsOn()
        }
    } else {
        // It's day time
        
        if light_state {
            // If lights are on, turn them off
            turnLightsOff()
        }
    }
}

// This function gets the current hour in 24-hour format.
function getCurrentHour() {
    // Implementation depends on the environment or system
}

// This function gets the current state of the lights.
function getLightState() {
    // Implementation depends on the environment or system
}

// This function turns the lights on.
function turnLightsOn() {
    // Implementation depends on the environment or system
}

// This function turns the lights off.
function turnLightsOff() {
    // Implementation depends on the environment or system
}
```

In this pseudo-code:
- We define fixed times for sunrise and sunset to determine the transition between day and night.
- We use helper functions, such as `getCurrentHour()`, to get the current time, and `getLightState()`, to check whether the lights are currently on or off.
- Based on whether it's currently day or night, and the current state of the lights, we make a decision to either turn the lights on or off by invoking either the `turnLightsOn()` or `turnLightsOff()` function. 

The implementation details of functions like `getCurrentHour()`, `getLightState()`, `turnLightsOn()`, and `turnLightsOff()` will depend on the specific environment or system you're working with, such as using APIs for smart home devices.